# Get Pretrained Models
Okay

In [15]:
!mkdir -p checkpoints/

!wget https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_s0.pt -P checkpoints

--2025-04-30 18:31:27--  https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_s0.pt
Resolving docs-assets.developer.apple.com (docs-assets.developer.apple.com)... 17.253.73.202, 17.253.73.201
Connecting to docs-assets.developer.apple.com (docs-assets.developer.apple.com)|17.253.73.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215934653 (206M) [application/octet-stream]
Saving to: ‘checkpoints/mobileclip_s0.pt’

mobileclip_s0.pt    100%[===================>] 205,93M  5,49MB/s    in 39s     

2025-04-30 18:32:07 (5,34 MB/s) - ‘checkpoints/mobileclip_s0.pt’ saved [215934653/215934653]



# Create the models

In [ ]:
!pip install torch
!pip install torchvision
!pip install timm
!pip install open-clip-torch
!pip install datasets
!pip install clip-benchmark

In [1]:
import torch
from PIL import Image
import mobileclip

model, _, preprocess = mobileclip.create_model_and_transforms('mobileclip_s0', pretrained='/Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/checkpoints/mobileclip_s0.pt')
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')

image = preprocess(Image.open("pngwing.com(1).png").convert('RGB')).unsqueeze(0)
text = tokenizer(["a purple dog", "a white dog", "a black dog"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# Set the print options for PyTorch to avoid scientific notation and limit decimal places
torch.set_printoptions(sci_mode=False, precision=4)

print("Label probs:", text_probs)

/opt/anaconda3/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/pngwing.com(1).png'

In [2]:
print(model.get_positional_embedding() )

def get_positional_embedding(self, lambda2: int = 4):
    """
    Get modified positional embedding for text encoder based on the given formula.
    """
    pos_embed = self.text_encoder.get_positional_embedding().pos_embed.pos_embed
    if pos_embed is None:
        raise ValueError("Positional embedding not found in text encoder.")

    max_pos, embed_dim = pos_embed.shape[2], pos_embed.shape[3]
    modified_pos_embed = torch.zeros((1, 1, max_pos, embed_dim), device=pos_embed.device)

    for pos in range(max_pos):
        if pos <= 20:
            modified_pos_embed[:, :, pos, :] = pos_embed[:, :, pos, :]
        else:
            lower_idx = pos // lambda2
            upper_idx = min(lower_idx + 1, max_pos - 1)  # Ensure upper_idx is within bounds
            alpha = (pos % lambda2) / lambda2
            modified_pos_embed[:, :, pos, :] = (1 - alpha) * pos_embed[:, :, lower_idx, :] + alpha * pos_embed[:, :, upper_idx, :]
    # turn the torch tensor into nn parameter
    modified_pos_embed = torch.nn.Parameter(modified_pos_embed, requires_grad=False)
    return modified_pos_embed

# Example usage
lambda2 = 4
new_pos_embed = get_positional_embedding(model, lambda2)
print("Modified Positional Embedding:", new_pos_embed)

# set the models pos embedding to the new one
model.text_encoder.get_positional_embedding().pos_embed.pos_embed = new_pos_embed


LearnablePositionalEmbedding(num_embeddings=77, embedding_dim=512, padding_idx=None)
Modified Positional Embedding: Parameter containing:
tensor([[[[     0.0000,      0.0000,      0.0000,  ...,      0.0000,
                0.0000,      0.0000],
          [     0.0041,      0.0016,     -0.0007,  ...,      0.0007,
                0.0036,     -0.0076],
          [     0.0077,      0.0026,      0.0012,  ...,      0.0001,
                0.0013,     -0.0043],
          ...,
          [     0.0075,      0.0051,      0.0000,  ...,      0.0011,
                0.0001,     -0.0003],
          [     0.0031,      0.0068,     -0.0008,  ...,      0.0024,
               -0.0002,     -0.0025],
          [    -0.0013,      0.0086,     -0.0016,  ...,      0.0036,
               -0.0004,     -0.0046]]]])


In [10]:
image = preprocess(Image.open("pngwing.com(1).png").convert('RGB')).unsqueeze(0)
text = tokenizer(["a purple dog", "a white dog", "a black dog"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# Set the print options for PyTorch to avoid scientific notation and limit decimal places
torch.set_printoptions(sci_mode=False, precision=4)

print("Label probs:", text_probs)

Label probs: tensor([[0.7638, 0.0161, 0.2201]])


In [ ]:
# download the coco dataset
# Download a smaller dataset, such as CIFAR-10
!mkdir -p cifar10
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -P cifar10/
!tar -xvzf cifar10/cifar-10-python.tar.gz -C cifar10/

In [ ]:
#  train the model using contrastive loss
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm
from torchvision import datasets
from custom_dataset import CustomDataset

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Load the CIFAR-10 dataset
cifar10_dataset = datasets.CIFAR10(root='cifar10', train=True, download=True)
# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, caption = self.dataset[idx]
        if self.transform:
            image = self.transform(image)
        return image, caption

# Create the dataset and dataloader
train_dataset = CustomDataset(cifar10_dataset, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=1)
# Define the contrastive loss function
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, image_features, text_features):
        # Compute the cosine similarity
        logits = torch.matmul(image_features, text_features.T) / self.temperature
        labels = torch.arange(logits.size(0)).to(logits.device)
        loss = nn.CrossEntropyLoss()(logits, labels)
        return loss
# Initialize the model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = ContrastiveLoss(temperature=0.07)
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, captions in tqdm(train_dataloader):
        
        images = images.to(device)
        captions = tokenizer(captions).to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            image_features = model.encode_image(images)
            text_features = model.encode_text(captions)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            loss = criterion(image_features, text_features)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_dataloader):.4f}")
    # Save the model checkpoint

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


# Create the dataset and dataloader
train_dataset = CustomDataset(cifar10_dataset, transform=preprocess)
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

# Replace `MobileCLIPModel()` with your Mobile CLIP implementation.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 3. Contrastive Loss (InfoNCE)
def contrastive_loss(image_embeds: torch.Tensor, text_embeds: torch.Tensor, temperature: float = 0.07) -> torch.Tensor:
    # Normalize embeddings
    image_embeds = F.normalize(image_embeds, dim=1)
    text_embeds = F.normalize(text_embeds, dim=1)
    # Compute logits
    logits = image_embeds @ text_embeds.t() / temperature
    labels = torch.arange(logits.size(0), device=logits.device)
    loss_i2t = F.cross_entropy(logits, labels)
    loss_t2i = F.cross_entropy(logits.t(), labels)
    return (loss_i2t + loss_t2i) / 2

# 4. Optimizer and training setup
optimizer = optim.Adam(model.parameters(), lr=1e-4)
epochs = 10

# 5. Training loop
def train():
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for images, captions in tqdm(dataloader):
            images = images.to(device)
            captions = tokenizer(captions).to(device)
            # Forward pass: user-provided model should return (image_embeds, text_embeds)
            image_embeds = model.encode_image(images)
            text_embeds = model.encode_text(captions)

            # Compute contrastive loss
            loss = contrastive_loss(image_embeds, text_embeds)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * images.size(0)

        avg_loss = total_loss / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

if __name__ == '__main__':
    train()


In [20]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

# Define the transformation pipeline
# Remove transforms.ToPILImage() since CIFAR-10 images are already PIL images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input size
    transforms.ToTensor(),         # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Load the CIFAR-10 dataset
cifar10_dataset = datasets.CIFAR10(root='cifar10', train=True, download=True)

# Apply the transformation pipeline to the images
images = [transform(img) for img, _ in cifar10_dataset]

# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
model.eval()

# Generate captions for a subset (e.g., first 100 images for demo)
captions = []
for image in images[:100]:  # Increase this number as needed
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    captions.append(caption)

# Custom dataset combining image and generated caption
class CIFAR10WithCaptions(Dataset):
    def __init__(self, images, captions, transform=None):
        self.images = images
        self.captions = captions
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        caption = self.captions[idx]
        if self.transform:
            img = self.transform(img)
        return img, caption

# Example usage
transform_tensor = transforms.ToTensor()
custom_dataset = CIFAR10WithCaptions(images[:100], captions, transform=transform_tensor)


Files already downloaded and verified


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

ValueError: The image to be converted to a PIL image contains values outside the range [0, 1], got [-2.1179039478302, 2.393557548522949] which cannot be converted to uint8.

In [22]:

# Example usage
transform_tensor = transforms.ToTensor()
custom_dataset = CIFAR10WithCaptions(images[:100], captions, transform=transform_tensor)

NameError: name 'CIFAR10WithCaptions' is not defined

In [21]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

# Get the first image and caption
img, caption = custom_dataset[0]

# Convert the tensor image to a PIL image for plotting
img = F.to_pil_image(img)

# Plot the image
plt.imshow(img)
plt.title(f"Caption: {caption}")
plt.axis('off')
plt.show()

NameError: name 'custom_dataset' is not defined